In [151]:
from github import Github, RateLimitExceededException
import os
import json

In [146]:
# Read Token
access_token = open('GITHUB_TOKEN.txt', 'r').read()
user = Github(access_token)
user.per_page = 100


def check_rate_limit():
    rate_limit = user.get_rate_limit()
    remaining = rate_limit.search.remaining
    
    if remaining > 0:
        return
    
    reset_time = rate_limit.search.reset
    now = datetime.utcnow()

    if now < reset_time:
        sleep_time = (reset_time - now).total_seconds()
        print('=> Waiting {} seconds for rate limit reset.'.format(sleep_time))
        time.sleep(sleep_time)
        
# Number of Results = NumberOfRequests * 10 pages * 100 resultsPerPage
def search(language, order, size, filename):
    
    repo_list = []
    size_query = ', size:' + size
    
    try:
        repos = user.search_repositories(query='language:'+ language + size_query, sort='stars', order=order)
            
        # Iterate through pages
        for j in range (0,10):
                
            # Iterate through repositories of a page
            for elementOfPage in repos.get_page(j):
                    
                repo_list.append({
                    'id': elementOfPage.id,
                    'owner': elementOfPage.owner.login,
                    'name': elementOfPage.name,
                    'language': language,
                    'stars': elementOfPage.stargazers_count,
                    'forks': elementOfPage.forks_count,
                    'default_branch': elementOfPage.default_branch,
                })
               
    except RateLimitExceededException:
        check_rate_limit()

            
    with open(filename, 'w') as f:
        f.write(json.dumps(repo_list, indent=4))

In [150]:
search('Java', 'asc', '50000..100000', './data/java_worst.json')